# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
data = pd.read_csv('../Cities_Weather_Info.csv')
data = data.dropna()
data

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,City Date/Time
0,0,komsomolskiy,40.4272,71.7189,23.40,40.0,100.0,1.03,UZ,1.651054e+09
1,1,lebu,-37.6167,-73.6500,11.07,84.0,85.0,9.86,CL,1.651054e+09
2,2,eyl,7.9803,49.8164,31.06,57.0,83.0,5.40,SO,1.651054e+09
3,3,oga,46.4631,10.3470,11.56,64.0,6.0,1.97,IT,1.651054e+09
4,4,ushuaia,-54.8000,-68.3000,1.58,97.0,98.0,3.77,AR,1.651054e+09
...,...,...,...,...,...,...,...,...,...,...
559,559,coihaique,-45.5752,-72.0662,-1.03,93.0,0.0,1.54,CL,1.651054e+09
560,560,robertsport,6.7533,-11.3686,28.26,75.0,97.0,1.89,LR,1.651054e+09
561,561,novoagansk,61.9449,76.6625,5.07,100.0,100.0,5.92,RU,1.651054e+09
562,562,mareeba,-17.0000,145.4333,23.40,88.0,60.0,1.42,AU,1.651054e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
#store latitude and longitude as locations
locations = data[["Latitude", "Longitude"]]
humidity = data["Humidity"]

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}


fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False,max_intensity=max(data["Humidity"]),
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:

ideal_dataframe = data.loc[data['Humidity'] > 30]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Humidity'] < 70]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Max Temp'] < 32]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Max Temp'] > 28]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Cloudiness'] < 30]
ideal_dataframe


,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,City Date/Time
48,48,akyab,20.1500,92.9000,30.19,68.0,1.0,7.00,MM,1.651054e+09
175,175,koungou,-12.7336,45.2042,30.84,66.0,20.0,5.14,YT,1.651054e+09
323,323,katherine,-14.4667,132.2667,31.18,42.0,0.0,2.57,AU,1.651054e+09
393,393,cap malheureux,-19.9842,57.6142,28.59,64.0,25.0,3.13,MU,1.651054e+09
419,419,kununurra,-15.7667,128.7333,29.99,51.0,5.0,3.60,AU,1.651054e+09
436,436,oussouye,12.4850,-16.5469,28.95,32.0,28.0,4.53,SN,1.651054e+09
486,486,bissau,11.8636,-15.5977,29.72,35.0,22.0,3.81,GW,1.651054e+09
493,493,ahvaz,31.3203,48.6693,31.97,35.0,0.0,4.12,IR,1.651054e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
hotel_df = ideal_dataframe
hotel_df["Hotel Name"] = ""

params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():

    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City Name"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    hotel_name = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    hotel_name = hotel_name.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print(f"Could not find a hotel in {city}")

# Visualize to confirm lat lng appear
hotel_df

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,City Date/Time,Hotel Name
48,48,akyab,20.1500,92.9000,30.19,68.0,1.0,7.00,MM,1.651054e+09,Hotel Memory
175,175,koungou,-12.7336,45.2042,30.84,66.0,20.0,5.14,YT,1.651054e+09,Perle du Sud
323,323,katherine,-14.4667,132.2667,31.18,42.0,0.0,2.57,AU,1.651054e+09,Pine Tree Motel
393,393,cap malheureux,-19.9842,57.6142,28.59,64.0,25.0,3.13,MU,1.651054e+09,The Ravenala Attitude Hotel
419,419,kununurra,-15.7667,128.7333,29.99,51.0,5.0,3.60,AU,1.651054e+09,Hotel Kununurra
436,436,oussouye,12.4850,-16.5469,28.95,32.0,28.0,4.53,SN,1.651054e+09,La Tortue Bleue
486,486,bissau,11.8636,-15.5977,29.72,35.0,22.0,3.81,GW,1.651054e+09,Hotel Ancar
493,493,ahvaz,31.3203,48.6693,31.97,35.0,0.0,4.12,IR,1.651054e+09,tashrifat tourism travel agency


In [40]:

hotel_df = hotel_df.rename(columns = {"City Name": "City"})

In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [42]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…